## Download model

It's important that your model have this directory structure for Triton Inference Server to be able to load it. [Read more about the directory structure that Triton expects](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html).

In [ ]:
!pip install azure-storage-blob

In [ ]:
import os
import sys
from pathlib import Path
from src.model_utils import download_triton_models, delete_triton_models

prefix = Path(".")
download_triton_models(prefix)

## Register model

The registered model should follow the Triton specified model folder structure for Triton Inference Server to be able to load it.

In [ ]:
subscripton = "subscription_id"
resource_group = "resource_group"
workspace = "workspace"
model_name = "densenet-onnx"
endpoint_name = "lite14"

In [ ]:
!az ml model create -n $model_name -v 1 -l models -g $resource_group -w $workspace --subscription $subscripton

In [ ]:
!az ml model show -n $model_name -v 1 -g $resource_group -w $workspace --subscription $subscripton

## Create endpoint

Deploy to a [pre-created AKS compute](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-kubernetes?tabs=python#create-a-new-aks-cluster) aks-test

In [ ]:
!az ml endpoint create -g $resource_group -w $workspace -n $endpoint_name overwrite -f endpoint-aks.yml

In [ ]:
!az ml endpoint show -g $resource_group -w $workspace -n $endpoint_name

## Test Webservice

Get scoring URI and auth token

In [ ]:
!az ml endpoint show -n $endpoint_name --query "scoring_uri"

In [ ]:
!az ml endpoint list-keys -g $resource_group -w $workspace -n $endpoint_name

In [ ]:
import requests

service_key = "service_key"
headers = {}
headers["Authorization"] = f"Bearer {service_key}"

# Check the state of server.
service_url = "service_url"
requests.get(f"{service_url}/v2/health/ready", headers=headers)

In [ ]:
# Check the status of model.
resp = requests.post(f"{service_url}/v2/repository/index", headers=headers)
print(resp.text)

In [ ]:
# Check metadata of model for inference 
resp = requests.get(f"{service_url}/v2/models/densenet_onnx", headers=headers)
print(resp.text)